In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",15)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels_20percent.pkl")
    kdd_train_2labels_y = pd.read_pickle("dataset/kdd_train_2labels_y_20percent.pkl")
    
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels_20percent.pkl")
    kdd_test_2labels_y = pd.read_pickle("dataset/kdd_test_2labels_y_20percent.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels_20percent.pkl")
    kdd_train_5labels_y = pd.read_pickle("dataset/kdd_train_5labels_y_20percent.pkl")
    
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels_20percent.pkl")
    kdd_test_5labels_y = pd.read_pickle("dataset/kdd_test_5labels_y_20percent.pkl")
    

In [3]:
import collections
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import metrics
from sklearn.decomposition import SparsePCA

In [4]:
pca = SparsePCA(n_components=40)

In [5]:
names_noscaling = ["Decision Tree", "Random Forest"]
classifiers_noscaling = [
                DecisionTreeClassifier(),
                RandomForestClassifier()
                ]

names_withscaling = ["SVC", "Non - Linear SVM", "AdaBoost", "Naive Bayes"]
classifiers_withscaling = [SVC(), NuSVC(),
                AdaBoostClassifier(), GaussianNB()]

score = collections.namedtuple("score", ["name", "valid_score" ,"test_score"])
scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

x_train, y_train = dataset.kdd_train_2labels.iloc[:,:-2], dataset.kdd_train_2labels_y
x_test, y_test = dataset.kdd_test_2labels.iloc[:,:-2], dataset.kdd_test_2labels_y

x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)

In [6]:
x_train.shape

(25192, 40)

In [7]:
x_test.shape

(11850, 40)

In [8]:
for name, clf in zip(names_noscaling, classifiers_noscaling):
    print("Classifier: {}".format(name))
    
    clf_p = make_pipeline(clf)    
    valid_score = cross_val_score(clf_p, x_train, y_train, cv=cv)
    
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    test_acc = metrics.accuracy_score(y_test, y_pred) 
    
    scores.append(score(name, valid_score.mean(), test_acc))
    print("Score: {} \n".format(test_acc))

for name, clf in zip(names_withscaling, classifiers_withscaling):
    print("Classifier: {}".format(name))
    
    clf_p = make_pipeline(preprocessing.StandardScaler(), clf)    
    valid_score = cross_val_score(clf_p, x_train, y_train, cv=cv)
    
    scaler = preprocessing.StandardScaler().fit(x_train)
    clf.fit(scaler.transform(x_train), y_train)
    y_pred = clf.predict(scaler.transform(x_test))
    test_acc = metrics.accuracy_score(y_test, y_pred) 
    
    scores.append(score(name, valid_score.mean(), test_acc))
    print("Score: {} \n".format(test_acc))

Classifier: Decision Tree
Score: 0.5959493670886076 

Classifier: Random Forest
Score: 0.6254852320675105 

Classifier: SVC
Score: 0.6270886075949367 

Classifier: Non - Linear SVM
Score: 0.8183966244725739 

Classifier: AdaBoost
Score: 0.6545147679324894 

Classifier: Naive Bayes
Score: 0.8183966244725739 



In [9]:
pd.DataFrame(scores)

,name,valid_score,test_score
0,Decision Tree,0.994761,0.595949
1,Random Forest,0.995614,0.625485
2,SVC,0.990633,0.627089
3,Non - Linear SVM,0.924886,0.818397
4,AdaBoost,0.989720,0.654515
5,Naive Bayes,0.923536,0.818397


In [10]:
#Naive Bayes